# Train Emotion Recognition Model

Here we'll train a emotion recognition model, using the output data from the sentiment analysis.

In [2]:
# Add project path to the PYTHONPATH

import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())

## Load Dataset

Load the emotion labeled dataset

In [3]:
from pathlib import Path
import pandas as pd
from nlp import Dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladislavklyuev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
dataset_path = Path('../datasets/sentiment_analysis/dataset.csv').resolve()
dataset = Dataset(dataset_path)
dataset.load()
dataset.preprocess_texts()

Time to clean up: 23.21 sec


In [5]:
dataset.cleaned_data.head()

,label,text
0,joy,thanks also constantly reminding us army guide...
1,joy,oh thank sharing love seeing pictures like i'v...
2,joy,i'm contemplating certain things know god help...
3,joy,hey drew :hushed_face: hope feel better real s...
4,joy,one fav jokes :grinning_face_with_smiling_eyes...


## Tokenize

Transform the text corpus to a vector representation

- **num_words**: Number of words to use

In [6]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

In [7]:
num_words = 10000

tokenizer = Tokenizer(num_words=num_words, lower=True)
tokenizer.fit_on_texts(dataset.cleaned_data.text)

file_to_save = Path('../datasets/sentiment_analysis/tokenizer.pickle').resolve()
with file_to_save.open('wb') as file:
    pickle.dump(tokenizer, file)

## Split data

Split the dataset in train and validation data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
data = dataset.cleaned_data.copy()

train = pd.DataFrame(columns=['label', 'text'])
validation = pd.DataFrame(columns=['label', 'text'])
for label in data.label.unique():
    label_data = data[data.label == label]
    train_data, validation_data = train_test_split(label_data, test_size=0.3)
    train = pd.concat([train, train_data])
    validation = pd.concat([validation, validation_data])

## Model

Define the **LSTM** + **CNN** model

In [11]:
from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Conv1D, Dense, concatenate
from tensorflow.keras.models import Model

In [13]:
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
num_classes = len(data.label.unique())
embedding_dim = 500
input_length = 100
lstm_units = 128
lstm_dropout = 0.1
recurrent_dropout = 0.1
spatial_dropout=0.2
filters=64
kernel_size=3

In [15]:
input_layer = Input(shape=(input_length,))
output_layer = Embedding(
  input_dim=input_dim,
  output_dim=embedding_dim,
  input_shape=(input_length,)
)(input_layer)

output_layer = SpatialDropout1D(spatial_dropout)(output_layer)

output_layer = Bidirectional(
LSTM(lstm_units, return_sequences=True,
     dropout=lstm_dropout, recurrent_dropout=recurrent_dropout)
)(output_layer)
output_layer = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                    kernel_initializer='glorot_uniform')(output_layer)

avg_pool = GlobalAveragePooling1D()(output_layer)
max_pool = GlobalMaxPooling1D()(output_layer)
output_layer = concatenate([avg_pool, max_pool])

output_layer = Dense(num_classes, activation='softmax')(output_layer)

model = Model(input_layer, output_layer)

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 500)     5000000     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 500)     0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 256)     644096      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

## Prepare the data

Prepare the model input data

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

In [21]:
train_sequences = [text.split() for text in train.text]
validation_sequences = [text.split() for text in validation.text]
list_tokenized_train = tokenizer.texts_to_sequences(train_sequences)
list_tokenized_validation = tokenizer.texts_to_sequences(validation_sequences)
x_train = pad_sequences(list_tokenized_train, maxlen=input_length)
x_validation = pad_sequences(list_tokenized_validation, maxlen=input_length)

encoder = LabelBinarizer()
encoder.fit(data.label.unique())

encoder_path = Path('../models/emotion_recognition', 'encoder.pickle')
with encoder_path.open('wb') as file:
    pickle.dump(encoder, file)

y_train = encoder.transform(train.label)
y_validation = encoder.transform(validation.label)

## Train model

Do the training process with the given data

In [23]:
batch_size = 128
epochs = 1

In [25]:
model.fit(
    x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_validation, y_validation)
)

Train on 41581 samples, validate on 17822 samples
41581/41581 [==============================] - 355s 9ms/sample - loss: 0.5735 - accuracy: 0.7475 - val_loss: 0.4302 - val_accuracy: 0.8133


In [27]:
model_file = Path('../models/emotion_recognition/model_weights.h5').resolve()
model.save_weights(model_file.as_posix())